<center><img src="https://drive.google.com/uc?export=view&id=1f0tvL6DLzfOQ_xHNdaBe_6fDUd27lc6U" alt="PSUT"  width="180px"> 
<p> 
King Hussein School for Computing Sciences <br>
Department of Data Science <br>

<b>Big Data 2020-2021 </b> 
</p></center>


<hr>
<h4><b>Grading </b></h4>

<table border>
<tr>
<td> <b>Question</b></td>
<td> <b>Mark</b></td>
</tr>

<tr>
<td> <b>Q1</b></td>
<td>  /2</td>
</tr>

<tr>
<td> <b>Q2</b></td>
<td>  /2</td>
</tr>

<tr>
<td> <b>Q3</b></td>
<td>  /2</td>
</tr>

<tr>
<td> <b>Q4</b></td>
<td>  /4</td>
</tr>

<td> <b>Total</b></td>
<th colspan=3> /10</th> 
</tr>
</table>



<h4><b>Programming Assignment Guidelines</b></h4>
<ul>


<li>  The assignment should be performed individually.
<li>  Cutoff date for this homeworkis 19/01/2021 @ 23:59
<li>  Any late assignment will not be accepted.
<li>  Academic Fraud: Cases of plagiarism will be dealt with according to university regulations.
<li>  Your homework should be written in ajupyter notebook file
<li>  Show all code in each step.
</ul>


### Student Name: Tamer Tahamoqa
### Student ID: 20208019


# Questions

## The Dataset file contains 9 columns (A,B,C,.., H, Class). Note that the data is ";" seperated file. Download the dataset file from E-learning System or use this [Link](https://drive.google.com/uc?export=download&id=1r4SnJcvE3x0YMqeiB3uAtevegKO6MbNr) then use Spark to load the dataset as a spark DataFrame. 

**Q1 *Min-max normalization*:**

> Use Spark API to transform all numeric columns using Min-Max scaling approach.Then store all normalized columns except the class column in a separate file (name it NormalizedFile.csv), the file should include 8 columns.

**Q2. *Z-score normalization*:**

> Use Spark API to transform all numeric columns using z-score approach. Then store all normalized columns except the class column in a separate file (name it ZSCORENormalizedFile.csv), the file should include 8 columns.

**Q3. *K-means* on Normalized and Scaled features:**

> Use Spark API to to train kmeans clustering model to cluster the dataset file (NormalizedFile.csv) and then the dataset file (ZSCORENormalizedFile.csv) by using k=2 for each model, and display the cluster number for all rows in both cases. Compare the result for clustering ZSCORENormalizedFile.csv and NormalizedFile.csv. Can Identify which model did better and why? 


**Q4. *KNN* classifier.** 


> Use Spark API to train a KNN model on 75% of the Dataset and provide accuracy and confusion matrix using 25% of the Dataset. Compare the performance of the classifer based on Normalized and Scaled features in NormalizedFile.csv and ZSCORENormalizedFile.csv




In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.master("local[*]").getOrCreate()

### Reading and Analyzing the dataset

In [2]:
df = spark.read.csv("Dataset.csv", header=True, inferSchema=True, sep=';')

In [3]:
df.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: integer (nullable = true)
 |-- D: integer (nullable = true)
 |-- E: integer (nullable = true)
 |-- F: double (nullable = true)
 |-- G: double (nullable = true)
 |-- H: integer (nullable = true)
 |-- Class: string (nullable = true)



In [4]:
df.summary().show()

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+
|summary|                 A|                B|                 C|                 D|                 E|                 F|                 G|                 H|          Class|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+
|  count|               768|              768|               768|               768|               768|               768|               768|               768|            768|
|   mean|3.8450520833333335|     120.89453125|       69.10546875|20.536458333333332| 79.79947916666667|31.992578124999977|0.4718763020833327|33.240885416666664|           null|
| stddev|  3.36957806269887|31.97261819513622|19.355807170644777|15.952217567727642|115.24400235133803| 7.884160320

In [5]:
df.show()

+---+---+---+---+---+----+-----+---+---------------+
|  A|  B|  C|  D|  E|   F|    G|  H|          Class|
+---+---+---+---+---+----+-----+---+---------------+
|  6|148| 72| 35|  0|33.6|0.627| 50|tested_positive|
|  1| 85| 66| 29|  0|26.6|0.351| 31|tested_negative|
|  8|183| 64|  0|  0|23.3|0.672| 32|tested_positive|
|  1| 89| 66| 23| 94|28.1|0.167| 21|tested_negative|
|  0|137| 40| 35|168|43.1|2.288| 33|tested_positive|
|  5|116| 74|  0|  0|25.6|0.201| 30|tested_negative|
|  3| 78| 50| 32| 88|31.0|0.248| 26|tested_positive|
| 10|115|  0|  0|  0|35.3|0.134| 29|tested_negative|
|  2|197| 70| 45|543|30.5|0.158| 53|tested_positive|
|  8|125| 96|  0|  0| 0.0|0.232| 54|tested_positive|
|  4|110| 92|  0|  0|37.6|0.191| 30|tested_negative|
| 10|168| 74|  0|  0|38.0|0.537| 34|tested_positive|
| 10|139| 80|  0|  0|27.1|1.441| 57|tested_negative|
|  1|189| 60| 23|846|30.1|0.398| 59|tested_positive|
|  5|166| 72| 19|175|25.8|0.587| 51|tested_positive|
|  7|100|  0|  0|  0|30.0|0.484| 32|tested_pos

In [6]:
# Counting number of missing (nan) values in each column, used this blogpost as a reference:
#  https://www.datasciencemadesimple.com/count-of-missing-nanna-and-null-values-in-pyspark/

df.select([F.count(F.when(F.isnan(col), col)).alias(col) for col in df.columns]).show()

+---+---+---+---+---+---+---+---+-----+
|  A|  B|  C|  D|  E|  F|  G|  H|Class|
+---+---+---+---+---+---+---+---+-----+
|  0|  0|  0|  0|  0|  0|  0|  0|    0|
+---+---+---+---+---+---+---+---+-----+



In [7]:
# Counting number of null values in each column, used this blogpost as a reference:
#  https://www.datasciencemadesimple.com/count-of-missing-nanna-and-null-values-in-pyspark/

df.select([F.count(F.when(F.isnull(col), col)).alias(col) for col in df.columns]).show()

+---+---+---+---+---+---+---+---+-----+
|  A|  B|  C|  D|  E|  F|  G|  H|Class|
+---+---+---+---+---+---+---+---+-----+
|  0|  0|  0|  0|  0|  0|  0|  0|    0|
+---+---+---+---+---+---+---+---+-----+



In [8]:
# Checking number of duplicate rows
#  Used this stackoverflow answer: https://stackoverflow.com/a/48554666
df.groupBy(df.columns) \
    .count() \
    .where(F.col('count') > 1) \
    .select(F.sum('count')) \
    .show()

+----------+
|sum(count)|
+----------+
|      null|
+----------+



### __Q1__. Min-max normalization:
Use Spark API to transform all numeric columns using Min-Max scaling approach.Then store all normalized columns except the class column in a separate file (name it NormalizedFile.csv), the file should include 8 columns.

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler

numerical_columns = ["A", "B", "C", "D", "E", "F", "G", "H"]

# Relied on this StackOverflow answer on to run MinMaxScaler on multiple columns
#  to keep the 8 columns intact: https://stackoverflow.com/a/60281624
vector_assemblers = [
    VectorAssembler(inputCols=[column], outputCol=column + "_vector")
    for column in numerical_columns
]

minmax_scalers = [
    MinMaxScaler(inputCol=column + "_vector", outputCol=column + "_scaled")
    for column in numerical_columns
]

pipeline = Pipeline(stages=vector_assemblers + minmax_scalers)
scaler_model = pipeline.fit(df)

minmax_scaled_df = scaler_model.transform(df)

In [10]:
columns_to_drop = [column for column in numerical_columns] + [f'{column}_vector' for column in numerical_columns] + ['Class']
minmax_scaled_df = minmax_scaled_df.drop(*columns_to_drop)

In [11]:
minmax_scaled_df.printSchema()

root
 |-- A_scaled: vector (nullable = true)
 |-- B_scaled: vector (nullable = true)
 |-- C_scaled: vector (nullable = true)
 |-- D_scaled: vector (nullable = true)
 |-- E_scaled: vector (nullable = true)
 |-- F_scaled: vector (nullable = true)
 |-- G_scaled: vector (nullable = true)
 |-- H_scaled: vector (nullable = true)



In [12]:
minmax_scaled_df.show(truncate=False)

+----------------------+---------------------+---------------------+---------------------+---------------------+---------------------+----------------------+---------------------+
|A_scaled              |B_scaled             |C_scaled             |D_scaled             |E_scaled             |F_scaled             |G_scaled              |H_scaled             |
+----------------------+---------------------+---------------------+---------------------+---------------------+---------------------+----------------------+---------------------+
|[0.35294117647058826] |[0.7437185929648241] |[0.5901639344262295] |[0.35353535353535354]|[0.0]                |[0.5007451564828614] |[0.23441502988898377] |[0.48333333333333334]|
|[0.058823529411764705]|[0.4271356783919598] |[0.5409836065573771] |[0.29292929292929293]|[0.0]                |[0.3964232488822653] |[0.11656703672075147] |[0.16666666666666666]|
|[0.47058823529411764] |[0.9195979899497487] |[0.5245901639344263] |[0.0]                |[0.0]     

In [13]:
# Changing column data type from vector to double for pyspark csv write to occur without errors
#  https://stackoverflow.com/a/51842141
#  https://stackoverflow.com/a/46890719
#  https://stackoverflow.com/a/61009640
from pyspark.sql.types import DoubleType

cast_from_vector_to_double = F.udf(lambda x : x.toArray().tolist()[0], DoubleType())

for column_name in minmax_scaled_df.columns:
    minmax_scaled_df = minmax_scaled_df.withColumn(column_name, cast_from_vector_to_double(column_name))

In [14]:
minmax_scaled_df.show()

+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+
|            A_scaled|           B_scaled|           C_scaled|           D_scaled|           E_scaled|           F_scaled|            G_scaled|           H_scaled|
+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+
| 0.35294117647058826| 0.7437185929648241| 0.5901639344262295|0.35353535353535354|                0.0| 0.5007451564828614| 0.23441502988898377|0.48333333333333334|
|0.058823529411764705| 0.4271356783919598| 0.5409836065573771|0.29292929292929293|                0.0| 0.3964232488822653| 0.11656703672075147|0.16666666666666666|
| 0.47058823529411764| 0.9195979899497487| 0.5245901639344263|                0.0|                0.0|0.34724292101341286|  0.2536293766011956|0.18333333333333332|
|0.0588235294117

In [15]:
minmax_scaled_df.coalesce(1).write.csv('NormalizedFile.csv', header=True) # One resulting file in the resulting directory

#### __Q1. Answer__:
I have used the PySpark write method instead of converting to Pandas dataframe for practice.

### __Q2__. Z-score normalization:
Use Spark API to transform all numeric columns using z-score approach. Then store all normalized columns except the class column in a separate file (name it ZSCORENormalizedFile.csv), the file should include 8 columns.

In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

numerical_columns = ["A", "B", "C", "D", "E", "F", "G", "H"]

# Relied on this StackOverflow answer on to run MinMaxScaler on multiple columns
#  to keep the 8 columns intact: https://stackoverflow.com/a/60281624
vector_assemblers = [
    VectorAssembler(inputCols=[column], outputCol=column + "_vector")
    for column in numerical_columns
]

standard_scalers = [
    # Standardize to have unit standard deviation and zero mean
    StandardScaler(inputCol=column + "_vector", outputCol=column + "_scaled", withStd=True, withMean=True)
    for column in numerical_columns
]

pipeline = Pipeline(stages=vector_assemblers + standard_scalers)
scaler_model = pipeline.fit(df)

standard_scaled_df = scaler_model.transform(df)

In [17]:
columns_to_drop = [column for column in numerical_columns] + [f'{column}_vector' for column in numerical_columns] + ['Class']
standard_scaled_df = standard_scaled_df.drop(*columns_to_drop)

In [18]:
standard_scaled_df.printSchema()

root
 |-- A_scaled: vector (nullable = true)
 |-- B_scaled: vector (nullable = true)
 |-- C_scaled: vector (nullable = true)
 |-- D_scaled: vector (nullable = true)
 |-- E_scaled: vector (nullable = true)
 |-- F_scaled: vector (nullable = true)
 |-- G_scaled: vector (nullable = true)
 |-- H_scaled: vector (nullable = true)



In [19]:
standard_scaled_df.show(truncate=False)

+----------------------+----------------------+----------------------+----------------------+---------------------+----------------------+----------------------+-----------------------+
|A_scaled              |B_scaled              |C_scaled              |D_scaled              |E_scaled             |F_scaled              |G_scaled              |H_scaled               |
+----------------------+----------------------+----------------------+----------------------+---------------------+----------------------+----------------------+-----------------------+
|[0.6395304921176561]  |[0.84777132058967]    |[0.14954329852954068] |[0.9066790623472516]  |[-0.69243932472413]  |[0.2038799072674687]  |[0.46818687022979555] |[1.4250667195933635]   |
|[-0.8443348188985361] |[-1.1226647449053908] |[-0.16044119073007856]|[0.5305558070991296]  |[-0.69243932472413]  |[-0.6839762138098171] |[-0.36482303037766123]|[-0.19054772934660194] |
|[1.2330766165241331]  |[1.9424580236424815]  |[-0.2637693538166183] |

In [20]:
# Changing column data type from vector to double for pyspark csv write to occur without errors
#  https://stackoverflow.com/a/51842141
#  https://stackoverflow.com/a/46890719
#  https://stackoverflow.com/a/61009640
from pyspark.sql.types import DoubleType

cast_from_vector_to_double = F.udf(lambda x : x.toArray().tolist()[0], DoubleType())

for column_name in minmax_scaled_df.columns:
    standard_scaled_df = standard_scaled_df.withColumn(column_name, cast_from_vector_to_double(column_name))

In [21]:
standard_scaled_df.show()

+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|            A_scaled|            B_scaled|            C_scaled|            D_scaled|           E_scaled|            F_scaled|            G_scaled|            H_scaled|
+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|  0.6395304921176561|    0.84777132058967| 0.14954329852954068|  0.9066790623472516|  -0.69243932472413|  0.2038799072674687| 0.46818687022979555|  1.4250667195933635|
| -0.8443348188985361| -1.1226647449053908|-0.16044119073007856|  0.5305558070991296|  -0.69243932472413| -0.6839762138098171|-0.36482303037766123|-0.19054772934660194|
|  1.2330766165241331|  1.9424580236424815| -0.2637693538166183|   -1.28737325993346|  -0.69243932472413|  -1.102536956603395|  0.6040037018505766|-0.10551

In [22]:
standard_scaled_df.coalesce(1).write.csv('ZSCORENormalizedFile.csv', header=True) # One resulting file in the resulting directory

#### __Q2. Answer__:
I have used the PySpark write method instead of converting to Pandas dataframe for practice.

### __Q3__. K-means on Normalized and Scaled features:
Use Spark API to to train kmeans clustering model to cluster the dataset file (NormalizedFile.csv) and then the dataset file (ZSCORENormalizedFile.csv) by using k=2 for each model, and display the cluster number for all rows in both cases. Compare the result for clustering ZSCORENormalizedFile.csv and NormalizedFile.csv. Can Identify which model did better and why? 

##### __3.1- Min-max scaled data__

In [23]:
minmax_scaled_df = spark.read.csv("NormalizedFile.csv", inferSchema=True, header=True)

In [24]:
minmax_scaled_df.show()

+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+
|            A_scaled|           B_scaled|           C_scaled|           D_scaled|           E_scaled|           F_scaled|            G_scaled|           H_scaled|
+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+
| 0.35294117647058826| 0.7437185929648241| 0.5901639344262295|0.35353535353535354|                0.0| 0.5007451564828614| 0.23441502988898377|0.48333333333333334|
|0.058823529411764705| 0.4271356783919598| 0.5409836065573771|0.29292929292929293|                0.0| 0.3964232488822653| 0.11656703672075147|0.16666666666666666|
| 0.47058823529411764| 0.9195979899497487| 0.5245901639344263|                0.0|                0.0|0.34724292101341286|  0.2536293766011956|0.18333333333333332|
|0.0588235294117

In [25]:
minmax_scaled_df.printSchema()

root
 |-- A_scaled: double (nullable = true)
 |-- B_scaled: double (nullable = true)
 |-- C_scaled: double (nullable = true)
 |-- D_scaled: double (nullable = true)
 |-- E_scaled: double (nullable = true)
 |-- F_scaled: double (nullable = true)
 |-- G_scaled: double (nullable = true)
 |-- H_scaled: double (nullable = true)



In [26]:
# Creating 'features' vector column
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

vector_assembler = VectorAssembler(inputCols=minmax_scaled_df.columns, outputCol="features")

pipeline = Pipeline(stages=[vector_assembler])
pipeline_model = pipeline.fit(minmax_scaled_df)

minmax_scaled_df = pipeline_model.transform(minmax_scaled_df)

In [27]:
minmax_scaled_df.printSchema()

root
 |-- A_scaled: double (nullable = true)
 |-- B_scaled: double (nullable = true)
 |-- C_scaled: double (nullable = true)
 |-- D_scaled: double (nullable = true)
 |-- E_scaled: double (nullable = true)
 |-- F_scaled: double (nullable = true)
 |-- G_scaled: double (nullable = true)
 |-- H_scaled: double (nullable = true)
 |-- features: vector (nullable = true)



In [28]:
minmax_scaled_df.show()

+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+
|            A_scaled|           B_scaled|           C_scaled|           D_scaled|           E_scaled|           F_scaled|            G_scaled|           H_scaled|            features|
+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+
| 0.35294117647058826| 0.7437185929648241| 0.5901639344262295|0.35353535353535354|                0.0| 0.5007451564828614| 0.23441502988898377|0.48333333333333334|[0.35294117647058...|
|0.058823529411764705| 0.4271356783919598| 0.5409836065573771|0.29292929292929293|                0.0| 0.3964232488822653| 0.11656703672075147|0.16666666666666666|[0.05882352941176...|
| 0.47058823529411764| 0.9195979899497487| 0.5245901639344263|             

In [29]:
from pyspark.ml.clustering import KMeans

# Added random seed value 420 for reproducible results
kmeans = KMeans(
    featuresCol='features',
    predictionCol='predicted_cluster_id',
    k=2,
    tol=0.0001,
    maxIter=1000,
    seed=420,
    distanceMeasure='euclidean'
)

kmeans_model_minmax = kmeans.fit(minmax_scaled_df)

In [30]:
minmax_scaled_df = kmeans_model_minmax.transform(minmax_scaled_df)

In [31]:
minmax_scaled_df.show()

+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+
|            A_scaled|           B_scaled|           C_scaled|           D_scaled|           E_scaled|           F_scaled|            G_scaled|           H_scaled|            features|predicted_cluster_id|
+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+
| 0.35294117647058826| 0.7437185929648241| 0.5901639344262295|0.35353535353535354|                0.0| 0.5007451564828614| 0.23441502988898377|0.48333333333333334|[0.35294117647058...|                   0|
|0.058823529411764705| 0.4271356783919598| 0.5409836065573771|0.29292929292929293|                0.0| 0.3964232488822653| 0.11656703672075147|0.16666666666666666|[0.0588235294

##### __3.2- Z-score scaled data__

In [32]:
standard_scaled_df = spark.read.csv("ZSCORENormalizedFile.csv", inferSchema=True, header=True)

In [33]:
standard_scaled_df.show()

+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|            A_scaled|            B_scaled|            C_scaled|            D_scaled|           E_scaled|            F_scaled|            G_scaled|            H_scaled|
+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|  0.6395304921176561|    0.84777132058967| 0.14954329852954068|  0.9066790623472516|  -0.69243932472413|  0.2038799072674687| 0.46818687022979555|  1.4250667195933635|
| -0.8443348188985361| -1.1226647449053908|-0.16044119073007856|  0.5305558070991296|  -0.69243932472413| -0.6839762138098171|-0.36482303037766123|-0.19054772934660194|
|  1.2330766165241331|  1.9424580236424815| -0.2637693538166183|   -1.28737325993346|  -0.69243932472413|  -1.102536956603395|  0.6040037018505766|-0.10551

In [34]:
standard_scaled_df.printSchema()

root
 |-- A_scaled: double (nullable = true)
 |-- B_scaled: double (nullable = true)
 |-- C_scaled: double (nullable = true)
 |-- D_scaled: double (nullable = true)
 |-- E_scaled: double (nullable = true)
 |-- F_scaled: double (nullable = true)
 |-- G_scaled: double (nullable = true)
 |-- H_scaled: double (nullable = true)



In [35]:
# Creating 'features' vector column
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

vector_assembler = VectorAssembler(inputCols=standard_scaled_df.columns, outputCol="features")

pipeline = Pipeline(stages=[vector_assembler])
pipeline_model = pipeline.fit(standard_scaled_df)

standard_scaled_df = pipeline_model.transform(standard_scaled_df)

In [36]:
standard_scaled_df.printSchema()

root
 |-- A_scaled: double (nullable = true)
 |-- B_scaled: double (nullable = true)
 |-- C_scaled: double (nullable = true)
 |-- D_scaled: double (nullable = true)
 |-- E_scaled: double (nullable = true)
 |-- F_scaled: double (nullable = true)
 |-- G_scaled: double (nullable = true)
 |-- H_scaled: double (nullable = true)
 |-- features: vector (nullable = true)



In [37]:
standard_scaled_df.show()

+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|            A_scaled|            B_scaled|            C_scaled|            D_scaled|           E_scaled|            F_scaled|            G_scaled|            H_scaled|            features|
+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|  0.6395304921176561|    0.84777132058967| 0.14954329852954068|  0.9066790623472516|  -0.69243932472413|  0.2038799072674687| 0.46818687022979555|  1.4250667195933635|[0.63953049211765...|
| -0.8443348188985361| -1.1226647449053908|-0.16044119073007856|  0.5305558070991296|  -0.69243932472413| -0.6839762138098171|-0.36482303037766123|-0.19054772934660194|[-0.8443348188985...|
|  1.2330766165241331|  1.9424580236424815| -0.263

In [38]:
from pyspark.ml.clustering import KMeans

# Added random seed value 420 for reproducible results
kmeans = KMeans(
    featuresCol='features',
    predictionCol='predicted_cluster_id',
    k=2,
    tol=0.0001,
    maxIter=1000,
    seed=420,
    distanceMeasure='euclidean'
)

kmeans_model_standard = kmeans.fit(standard_scaled_df)

In [39]:
standard_scaled_df = kmeans_model_standard.transform(standard_scaled_df)

In [40]:
standard_scaled_df.show()

+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            A_scaled|            B_scaled|            C_scaled|            D_scaled|           E_scaled|            F_scaled|            G_scaled|            H_scaled|            features|predicted_cluster_id|
+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0.6395304921176561|    0.84777132058967| 0.14954329852954068|  0.9066790623472516|  -0.69243932472413|  0.2038799072674687| 0.46818687022979555|  1.4250667195933635|[0.63953049211765...|                   1|
| -0.8443348188985361| -1.1226647449053908|-0.16044119073007856|  0.5305558070991296|  -0.69243932472413| -0.6839762138098171|-0.36482303037766123|-0.190547

##### __3.3- Evaluating KMeans models using the Silhouette measure for clustering results__
Reference for using the PySpark ClusteringEvaluator is from the official documentation [here](https://spark.apache.org/docs/latest/ml-clustering.html#k-means) and [here](https://spark.apache.org/docs/2.3.0/api/python/pyspark.ml.html#pyspark.ml.evaluation.ClusteringEvaluator)

In [41]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Evaluate clustering by computing Silhouette score for Clustering results,
#  which expects two input columns: prediction and features. The metric computes the Silhouette 
#  measure using the squared Euclidean distance.

# The Silhouette is a measure for the validation of the consistency
#  within clusters. It ranges between 1 and -1, where a value close to
#  1 means that the points in a cluster are close to the other points
#  in the same cluster and far from the points of the other clusters.
evaluator = ClusteringEvaluator(
    predictionCol="predicted_cluster_id",
    featuresCol="features",
    metricName="silhouette"
)

In [42]:
silhouette_minmax = evaluator.evaluate(minmax_scaled_df)
print("Min-Max scaled data Silhouette measure with squared euclidean distance = {}\n".format(str(silhouette_minmax)))

wssse = kmeans_model_minmax.computeCost(minmax_scaled_df)
print("Min-Max scaled data Within set Sum of Squared Errors = {}\n".format(str(wssse)))

centers = kmeans_model_minmax.clusterCenters()
print("Cluster Centers for Min-Max scaled KMeans model: ")
for center in centers:
    print(center)

Min-Max scaled data Silhouette measure with squared euclidean distance = 0.41578027722961913

Min-Max scaled data Within set Sum of Squared Errors = 121.25848529922399

Cluster Centers for Min-Max scaled KMeans model: 
[0.43848137 0.66399728 0.61766051 0.17932311 0.08200296 0.48339578
 0.1697565  0.4249004 ]
[0.12310843 0.58008612 0.54157022 0.22108903 0.1003082  0.47358226
 0.16741382 0.09677627]


In [43]:
silhouette_standard = evaluator.evaluate(standard_scaled_df)
print("Standard (Z-Score) scaled data Silhouette measure with squared euclidean distance = {}\n".format(str(silhouette_standard)))

wssse = kmeans_model_standard.computeCost(standard_scaled_df)
print("Standard (Z-Score) scaled data Within set Sum of Squared Errors = {}\n".format(str(wssse)))

centers = kmeans_model_standard.clusterCenters()
print("Cluster Centers for Standard (Z-Score) scaled KMeans model: ")
for center in centers:
    print(center)

Standard (Z-Score) scaled data Silhouette measure with squared euclidean distance = 0.2758527960262051

Standard (Z-Score) scaled data Within set Sum of Squared Errors = 5149.290158623635

Cluster Centers for Standard (Z-Score) scaled KMeans model: 
[-0.05229762 -0.33485589 -0.20886229 -0.5447605  -0.45971271 -0.37122244
 -0.19806688 -0.0856377 ]
[0.07895915 0.50556674 0.3153411  0.82248154 0.69407606 0.5604731
 0.29904215 0.12929614]


#### __Q3. Answer__:
Since the Silhouette measure ranges between 1 and -1; where a value close to 1 means that the points in a cluster are close to other points in the same cluster and far from the points of the other clusters. __The greater silhouette measure and the lower Within Set Sum of Squared Error (WSSE) value for the KMeans clustering with the Min-Max scaled data would signify that it outperformed the KMeans clustering with the standard (z-score) scaled data__.

### __Q4__. KNN classifier:
Use Spark API to train a KNN model on 75% of the Dataset and provide accuracy and confusion matrix using 25% of the Dataset. Compare the performance of the classifer based on Normalized and Scaled features in NormalizedFile.csv and ZSCORENormalizedFile.csv

#### __Justification of using Sckikit-learn's KNearestNeighbors implementation instead of utilizing an implementation of KNearestNeighbors on Spark__


* Since the K-Nearest-Neighbours algorithm is not available in Spark as it would be unsuitable for Big Data since the KNN model is practically the entire dataset the model is being used on [reference](https://forums.databricks.com/answers/10558/view.html); I tried using this KNN [implementation](https://github.com/saurfang/spark-knn) on Spark which is available on GitHub. Package installation steps [here](https://spark-packages.org/package/saurfang/spark-knn).

    __Unfortunately__, the GitHub repository supports Spark version 2.1.0. I was having "'JavaPackage' object is not callable" errors  I have tried installing and running a version 2.1.0 Spark Session in this notebook but it appeared it did not have support for Python 3, so I installed a Python 2 kernel and ended up having the same problem. I have also tried Spark version 2.1.0 to 2.1.3.

For reference, the list of commands to download the required packages for the GitHub repository:

    ```
    !apt-get install -y wget
    !apt-get install -y git
    !mkdir knn-lib
    !git clone https://github.com/saurfang/spark-knn knn-lib/
    !cp knn_lib/python/pyspark_knn . -r
    !wget http://dl.bintray.com/spark-packages/maven/saurfang/spark-knn/0.3.0/spark-knn-0.3.0.jar
    !python knn-lib/python/test.py
    !wget https://archive.apache.org/dist/spark/spark-2.1.0/spark-2.1.0-bin-hadoop2.7.tgz
    !tar -xf spark-2.1.0-bin-hadoop2.7.tgz
    !wget https://archive.apache.org/dist/spark/spark-2.1.1/spark-2.1.1-bin-hadoop2.7.tgz
    !tar -xf spark-2.1.1-bin-hadoop2.7.tgz
    !wget https://archive.apache.org/dist/spark/spark-2.1.2/spark-2.1.2-bin-hadoop2.7.tgz
    !tar -xf spark-2.1.2-bin-hadoop2.7.tgz
    !wget https://archive.apache.org/dist/spark/spark-2.1.3/spark-2.1.3-bin-hadoop2.7.tgz
    !tar -xf spark-2.1.3-bin-hadoop2.7.tgz
    ```

For reference, the python code for importing the KNNClassifier implementation:

    ```
    import os
    os.environ["SPARK_HOME"] = "/opt/workspace/spark-2.1.1-bin-hadoop2.7"
    import findspark
    findspark.init("/opt/workspace/spark-2.1.1-bin-hadoop2.7")
    from pyspark.sql import SparkSession
    from pyspark.sql import functions as F
    from pyspark_knn.ml.classification import KNNClassifier

    spark_knn = SparkSession.builder.master("local[*]").config("spark.jars", "spark-knn-0.3.0.jar").getOrCreate()

    model = KNNClassifier()

    ```

    ```
    # Further testing using the test.py script from the GitHub repository:
    
    from pyspark import SparkContext
    from pyspark.sql import SQLContext
    from pyspark.mllib.linalg import Vectors
    from pyspark_knn.ml.classification import KNNClassifier

    # This is a simple test app. Use the following command to run:
    # spark-submit --driver-class-path ../spark-knn-core/target/scala-2.11/spark-knn_*.jar test.py

    sc = SparkContext(appName='test')
    sqlContext = SQLContext(sc)

    knn = KNNClassifier(k=1, topTreeSize=1, topTreeLeafSize=1, subTreeLeafSize=1, bufferSizeSampleSize=[1, 2, 3])  # bufferSize=-1.0,    
    ```

* I have also checked some models in PySpark ML that approximate Nearest Neighbors [link1](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.BucketedRandomProjectionLSHModel.approxNearestNeighbors) [link2](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.MinHashLSHModel.approxNearestNeighbors). __Unfortunately__, to my understanding, they don't act as a classifier for the purposes of the assignment.

__As per the reasons stated above and due to the lack of ability to allocate more time to find a way to make KNearestNeighbors work on Spark; I have decided to utilize the scikit-learn implementation of the algorithm.__

#### __4.1 Reading spark dataframes as pandas dataframes__

In [44]:
import sklearn
import pandas as pd
import numpy as np

In [45]:
pandas_df = df.toPandas()

In [46]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       768 non-null    int32  
 1   B       768 non-null    int32  
 2   C       768 non-null    int32  
 3   D       768 non-null    int32  
 4   E       768 non-null    int32  
 5   F       768 non-null    float64
 6   G       768 non-null    float64
 7   H       768 non-null    int32  
 8   Class   768 non-null    object 
dtypes: float64(2), int32(6), object(1)
memory usage: 36.1+ KB


In [47]:
pandas_df.head()

,A,B,C,D,E,F,G,H,Class
0,6,148,72,35,0,33.6,0.627,50,tested_positive
1,1,85,66,29,0,26.6,0.351,31,tested_negative
2,8,183,64,0,0,23.3,0.672,32,tested_positive
3,1,89,66,23,94,28.1,0.167,21,tested_negative
4,0,137,40,35,168,43.1,2.288,33,tested_positive


#### __4.2 One-hot encoding the class column__

In [48]:
pandas_df = pd.get_dummies(data=pandas_df, columns=['Class'], drop_first=True)

In [49]:
pandas_df.head()

,A,B,C,D,E,F,G,H,Class_tested_positive
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


#### __4.3 Splitting into training and test sets__

In [50]:
from sklearn.model_selection import train_test_split

seed = 420  # For reproduciblility

train_df, test_df = train_test_split(
    pandas_df,
    train_size=0.75,
    test_size=0.25,
    random_state=seed,  # For reproduciblility
    shuffle=False  # No random shuffling before splitting
)

In [51]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 575
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   A                      576 non-null    int32  
 1   B                      576 non-null    int32  
 2   C                      576 non-null    int32  
 3   D                      576 non-null    int32  
 4   E                      576 non-null    int32  
 5   F                      576 non-null    float64
 6   G                      576 non-null    float64
 7   H                      576 non-null    int32  
 8   Class_tested_positive  576 non-null    uint8  
dtypes: float64(2), int32(6), uint8(1)
memory usage: 27.6 KB


In [52]:
train_df.head()

,A,B,C,D,E,F,G,H,Class_tested_positive
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [53]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 576 to 767
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   A                      192 non-null    int32  
 1   B                      192 non-null    int32  
 2   C                      192 non-null    int32  
 3   D                      192 non-null    int32  
 4   E                      192 non-null    int32  
 5   F                      192 non-null    float64
 6   G                      192 non-null    float64
 7   H                      192 non-null    int32  
 8   Class_tested_positive  192 non-null    uint8  
dtypes: float64(2), int32(6), uint8(1)
memory usage: 9.2 KB


In [54]:
test_df.head()

,A,B,C,D,E,F,G,H,Class_tested_positive
576,6,108,44,20,130,24.0,0.813,35,0
577,2,118,80,0,0,42.9,0.693,21,1
578,10,133,68,0,0,27.0,0.245,36,0
579,2,197,70,99,0,34.7,0.575,62,1
580,0,151,90,46,0,42.1,0.371,21,1


#### __4.4 Applying Min-Max and ZSCORE scaling to the train and test sets, I have decided to apply scaling after splitting to make sure the train and test sets are the same for both scaling methods__

In [55]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler(with_mean=True, with_std=True)

columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

##### __4.4.1 Min-Max scaling__

In [56]:
# Was having issues when not using deep copies, the scaling transformation would
#  affect the original train_df and test_df if deep copying is not specified
minmax_train_df = train_df.copy(deep=True)
minmax_test_df = test_df.copy(deep=True)

minmax_train_df[columns] = minmax_scaler.fit_transform(minmax_train_df[columns])
minmax_test_df[columns] = minmax_scaler.fit_transform(minmax_test_df[columns])

In [57]:
minmax_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 575
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   A                      576 non-null    float64
 1   B                      576 non-null    float64
 2   C                      576 non-null    float64
 3   D                      576 non-null    float64
 4   E                      576 non-null    float64
 5   F                      576 non-null    float64
 6   G                      576 non-null    float64
 7   H                      576 non-null    float64
 8   Class_tested_positive  576 non-null    uint8  
dtypes: float64(8), uint8(1)
memory usage: 41.1 KB


In [58]:
minmax_train_df.head()

,A,B,C,D,E,F,G,H,Class_tested_positive
0,0.352941,0.747475,0.590164,0.555556,0.000000,0.500745,0.234415,0.483333,1
1,0.058824,0.429293,0.540984,0.460317,0.000000,0.396423,0.116567,0.166667,0
2,0.470588,0.924242,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333,1
3,0.058824,0.449495,0.540984,0.365079,0.111111,0.418778,0.038002,0.000000,0
4,0.000000,0.691919,0.327869,0.555556,0.198582,0.642325,0.943638,0.200000,1


In [59]:
minmax_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 576 to 767
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   A                      192 non-null    float64
 1   B                      192 non-null    float64
 2   C                      192 non-null    float64
 3   D                      192 non-null    float64
 4   E                      192 non-null    float64
 5   F                      192 non-null    float64
 6   G                      192 non-null    float64
 7   H                      192 non-null    float64
 8   Class_tested_positive  192 non-null    uint8  
dtypes: float64(8), uint8(1)
memory usage: 13.7 KB


In [60]:
minmax_test_df.head()

,A,B,C,D,E,F,G,H,Class_tested_positive
576,0.461538,0.363636,0.385965,0.202020,0.216667,0.418848,0.450031,0.285714,0
577,0.153846,0.433566,0.701754,0.000000,0.000000,0.748691,0.375543,0.000000,1
578,0.769231,0.538462,0.596491,0.000000,0.000000,0.471204,0.097455,0.306122,0
579,0.153846,0.986014,0.614035,1.000000,0.000000,0.605585,0.302297,0.836735,1
580,0.000000,0.664336,0.789474,0.464646,0.000000,0.734729,0.175667,0.000000,1


##### __4.4.2 ZSCORE scaling__

In [61]:
# Was having issues when not using deep copies, the scaling transformation would
#  affect the original train_df and test_df if deep copying is not specified
standard_train_df = train_df.copy(deep=True)
standard_test_df = test_df.copy(deep=True)

standard_train_df[columns] = standard_scaler.fit_transform(standard_train_df[columns])
standard_test_df[columns] = standard_scaler.fit_transform(standard_test_df[columns])

In [62]:
standard_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 575
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   A                      576 non-null    float64
 1   B                      576 non-null    float64
 2   C                      576 non-null    float64
 3   D                      576 non-null    float64
 4   E                      576 non-null    float64
 5   F                      576 non-null    float64
 6   G                      576 non-null    float64
 7   H                      576 non-null    float64
 8   Class_tested_positive  576 non-null    uint8  
dtypes: float64(8), uint8(1)
memory usage: 41.1 KB


In [63]:
standard_train_df.head()

,A,B,C,D,E,F,G,H,Class_tested_positive
0,0.655888,0.858193,0.165672,0.922316,-0.690469,0.212803,0.438215,1.429049,1
1,-0.839724,-1.075860,-0.145672,0.538462,-0.690469,-0.659347,-0.384206,-0.185769,0
2,1.254133,1.932667,-0.249454,-1.316832,-0.690469,-1.070503,0.572305,-0.100779,1
3,-0.839724,-0.953063,-0.145672,0.154608,0.121960,-0.472458,-0.932487,-1.035673,0
4,-1.138846,0.520502,-1.494832,0.922316,0.761532,1.396435,5.387642,-0.015788,1


In [64]:
standard_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 576 to 767
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   A                      192 non-null    float64
 1   B                      192 non-null    float64
 2   C                      192 non-null    float64
 3   D                      192 non-null    float64
 4   E                      192 non-null    float64
 5   F                      192 non-null    float64
 6   G                      192 non-null    float64
 7   H                      192 non-null    float64
 8   Class_tested_positive  192 non-null    uint8  
dtypes: float64(8), uint8(1)
memory usage: 13.7 KB


In [65]:
standard_test_df.head()

,A,B,C,D,E,F,G,H,Class_tested_positive
576,0.594081,-0.517118,-1.331274,-0.023507,0.444451,-1.119032,1.155826,0.136095,0
577,-0.569833,-0.182256,0.512029,-1.211204,-0.700388,1.430885,0.776148,-1.059408,1
578,1.757996,0.320038,-0.102406,-1.211204,-0.700388,-0.714283,-0.641316,0.221488,0
579,-0.569833,2.463158,0.000000,4.667900,-0.700388,0.324572,0.402798,2.441708,1
580,-1.151790,0.922790,1.024057,1.520501,-0.700388,1.322952,-0.242654,-1.059408,1


#### __4.5 Training on KNearestNeighbors with a set number of 7 neighbors on both min-max and zscore scaled data__

In [66]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import  accuracy_score, classification_report

X_columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
y_column = ['Class_tested_positive']

##### __4.5.1 min-max scaled data__

In [67]:
KNN_minmax = KNeighborsClassifier(
    n_neighbors=7,
    algorithm='auto',
    weights='uniform',
    n_jobs=-1  # use all available cpu cores
)

In [68]:
# Added .values.ravel() to y-column as there was a warning by scikit-learn with the following message:
#  DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
KNN_minmax.fit(X=minmax_train_df[X_columns], y=minmax_train_df[y_column].values.ravel())

KNeighborsClassifier(n_jobs=-1, n_neighbors=7)

In [69]:
y_pred_minmax = KNN_minmax.predict(X=minmax_test_df[X_columns])

In [70]:
accuracy = accuracy_score(y_true=minmax_test_df[y_column].values.ravel(), y_pred=y_pred_minmax) * 100
print(f'Accuracy score = {accuracy}%')

Accuracy score = 76.04166666666666%


In [71]:
print(classification_report(y_true=minmax_test_df[y_column].values.ravel(), y_pred=y_pred_minmax, digits=4))

              precision    recall  f1-score   support

           0     0.7533    0.9262    0.8309       122
           1     0.7857    0.4714    0.5893        70

    accuracy                         0.7604       192
   macro avg     0.7695    0.6988    0.7101       192
weighted avg     0.7651    0.7604    0.7428       192



##### __4.5.2 zscore scaled data__

In [72]:
KNN_standard = KNeighborsClassifier(
    n_neighbors=7,
    algorithm='auto',
    weights='uniform',
    n_jobs=-1  # use all available cpu cores
)

In [73]:
# Added .values.ravel() to y-column as there was a warning by scikit-learn with the following message:
#  DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
KNN_standard.fit(X=standard_train_df[X_columns], y=standard_train_df[y_column].values.ravel()) 

KNeighborsClassifier(n_jobs=-1, n_neighbors=7)

In [74]:
y_pred_standard = KNN_standard.predict(X=standard_test_df[X_columns])

In [75]:
accuracy = accuracy_score(y_true=standard_test_df[y_column].values.ravel(), y_pred=y_pred_standard) * 100
print(f'Accuracy score = {accuracy}%')

Accuracy score = 75.0%


In [76]:
print(classification_report(y_true=standard_test_df[y_column].values.ravel(), y_pred=y_pred_standard, digits=4))

              precision    recall  f1-score   support

           0     0.7681    0.8689    0.8154       122
           1     0.7037    0.5429    0.6129        70

    accuracy                         0.7500       192
   macro avg     0.7359    0.7059    0.7141       192
weighted avg     0.7446    0.7500    0.7416       192



#### __Q4. Answer__:
When setting the number of neighbours parameter for the KNN model to __7__; __the KNN model that was trained on min-max scaled data outperforms the KNN model trained on zscore scaled data__.

However, while experimenting with other values for the number of neighbours parameter, it appears that the __KNN model trained on the zscore scaled data generally tends to outperform the KNN model trained on the min-max scaled data__.